In [1]:
import pandas as pd
import numpy as np
import pymongo
import requests
import json
import pickle
from collections import defaultdict
from collections import Counter

In [9]:
client = pymongo.MongoClient("mongodb://localhost:37017" )
db=client["lol"]
coll=db["match_data"]

In [10]:
def lane_dicts(mongo_object):
    """

    """
    allied={}
    enemy={}
    winning_team=0
    bot_flag=[0,0]
    for i,player in enumerate(mongo_object["data"]["participants"]):

        if i<5:
            if bot_flag[0]==0 and player["timeline"]["lane"] == "BOTTOM":
                allied[player["timeline"]["lane"]+"2"]= player["championId"]
                bot_flag[0]=1
            else:
                allied[player["timeline"]["lane"]]= player["championId"]
        else:
            if bot_flag[1]==0 and player["timeline"]["lane"] == "BOTTOM":
                enemy[player["timeline"]["lane"]+"2"]= player["championId"]
                bot_flag[1]=1
            else:
                enemy[player["timeline"]["lane"]]= player["championId"]
    
            
    if mongo_object["data"]["teams"][0]["win"]=="Fail":
        winning_team=100
    else:
        winning_team=200
    return allied, enemy, winning_team

In [11]:
x=coll.find_one()

In [12]:
lane_dicts(x)

({u'BOTTOM': 21, u'BOTTOM2': 412, u'JUNGLE': 33, u'MIDDLE': 115, u'TOP': 57},
 {u'BOTTOM': 40, u'BOTTOM2': 15, u'JUNGLE': 59, u'MIDDLE': 91, u'TOP': 27},
 100)

In [21]:
def lane_matchups(mongo_object):
    allied,enemy,winning_team=lane_dicts(mongo_object)
    error_list=[]
    
    try:
        lane_matchups=Counter()
        lane_matchups_wins=Counter()
        for lane in allied:
            lane_matchups[(allied[lane],enemy[lane])]+=1
            if winning_team==100:
                lane_matchups_wins[(allied[lane],enemy[lane])]+=1
        lane_matchups[(allied["BOTTOM"],enemy["BOTTOM2"])]+=1
        lane_matchups[(allied["BOTTOM2"],enemy["BOTTOM"])]+=1
        if winning_team==100:
            lane_matchups_wins[(allied["BOTTOM"],enemy["BOTTOM2"])]+=1
            lane_matchups_wins[(allied["BOTTOM2"],enemy["BOTTOM"])]+=1

        for lane in enemy:
            lane_matchups[(enemy[lane],allied[lane])]+=1
            if winning_team==200:
                lane_matchups_wins[(enemy[lane],allied[lane])]+=1
        lane_matchups[(enemy["BOTTOM"],allied["BOTTOM2"])]+=1
        lane_matchups[(enemy["BOTTOM2"],allied["BOTTOM"])]+=1
        if winning_team==200:
            lane_matchups_wins[(enemy["BOTTOM"],allied["BOTTOM2"])]+=1
            lane_matchups_wins[(enemy["BOTTOM2"],allied["BOTTOM"])]+=1

        return lane_matchups, lane_matchups_wins, error_list
    except:
        
        error_list.append(mongo_object["_id"])
        return Counter({"666":1}),Counter({"666":1}),error_list
        

In [75]:
def total_and_wins_for_lane_matchup(iterable_games_data):
    """
    Calls and aggregates results of lane_matchups() over 
    iterable collection of games data.
    Returns two counter dicts
    """
    lane_total_vs=Counter()
    lane_wins_vs=Counter()
    errors=[]
    for game in iterable_games_data:
        lane_matchup, lane_matchup_wins, error = lane_matchups(game)
        lane_total_vs+=lane_matchup
        lane_wins_vs+=lane_matchup_wins
        errors.append(error)
    return lane_total_vs,lane_wins_vs, errors

In [65]:
for x in range(10):
    
    print list(coll.find({"_id":10}))[0]["data"]["participants"][x]["timeline"]["lane"]

MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE


In [69]:
import timeit

In [77]:
def time_me():
    return total_and_wins_for_lane_matchup(coll.find().limit(1000))

In [81]:
timeit.timeit(stmt=time_me,number=1)

9.290280818939209

In [76]:

total, wins, errors = total_and_wins_for_lane_matchup(coll.find().limit(50))

In [50]:
x=list(coll.find({"_id":65}))[0]

In [51]:
lanes=Counter()
for ch in x["data"]["participants"]:
    lanes[ch["timeline"]["lane"]]+=1
lanes["BOTTOM"]==4 and lanes["JUNGLE"] ==2 and lanes["TOP"] ==2 and lanes["MIDDLE"] ==2

False

In [126]:
coll.find_one()["data"]["participants"][0]["timeline"]["lane"]

u'JUNGLE'

In [127]:
def check_for_normal_lanes(gamedata):
    lanes=Counter()
    for champion in gamedata["data"]["participants"]:
        lanes[champion["timeline"]["lane"]]+=1
    return lanes["BOTTOM"]==4 and lanes["JUNGLE"] ==2 and lanes["TOP"] ==2 and lanes["MIDDLE"] ==2

In [56]:
for x in range(10,35):
    x=list(coll.find({"_id":65}))[0]
    lanes=Counter()
    for ch in x["data"]["participants"]:
        lanes[ch["timeline"]["lane"]]+=1
    print lanes["BOTTOM"]==4 and lanes["JUNGLE"] ==2 and lanes["TOP"] ==2 and lanes["MIDDLE"] ==2

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [28]:
len([x for x in errors if x!=[]])

65

In [24]:
sorted(total.values(),reverse=True)

[7,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [52]:
for x in list(coll.find({"_id":65}))[0]["data"]["participants"]:
    print x["timeline"]["lane"]

JUNGLE
MIDDLE
BOTTOM
JUNGLE
BOTTOM
JUNGLE
BOTTOM
JUNGLE
MIDDLE
JUNGLE


In [92]:
lane_matchups(x)

(Counter({(4, 245): 1,
          (18, 21): 1,
          (18, 25): 1,
          (21, 18): 1,
          (21, 267): 1,
          (25, 18): 1,
          (25, 267): 1,
          (64, 141): 1,
          (98, 122): 1,
          (122, 98): 1,
          (141, 64): 1,
          (245, 4): 1,
          (267, 21): 1,
          (267, 25): 1}),
 Counter({(4, 245): 1,
          (21, 18): 1,
          (21, 267): 1,
          (25, 18): 1,
          (25, 267): 1,
          (122, 98): 1,
          (141, 64): 1}))

In [155]:
def lane_matchups2(mongo_object):
    allied,enemy,winning_team=lane_dicts(mongo_object)
    
    if check_for_normal_lanes(mongo_object):
        lane_matchups=Counter()
        lane_matchups_wins=Counter()
        
        for lane in allied:
            lane_matchups[(allied[lane],enemy[lane])]+=1
            if winning_team==100:
                lane_matchups_wins[(allied[lane],enemy[lane])]+=1
        lane_matchups[(allied["BOTTOM"],enemy["BOTTOM2"])]+=1
        lane_matchups[(allied["BOTTOM2"],enemy["BOTTOM"])]+=1
        if winning_team==100:
            lane_matchups_wins[(allied["BOTTOM"],enemy["BOTTOM2"])]+=1
            lane_matchups_wins[(allied["BOTTOM2"],enemy["BOTTOM"])]+=1

        for lane in enemy:
            lane_matchups[(enemy[lane],allied[lane])]+=1
            if winning_team==200:
                lane_matchups_wins[(enemy[lane],allied[lane])]+=1
        lane_matchups[(enemy["BOTTOM"],allied["BOTTOM2"])]+=1
        lane_matchups[(enemy["BOTTOM2"],allied["BOTTOM"])]+=1
        if winning_team==200:
            lane_matchups_wins[(enemy["BOTTOM"],allied["BOTTOM2"])]+=1
            lane_matchups_wins[(enemy["BOTTOM2"],allied["BOTTOM"])]+=1

        return lane_matchups, lane_matchups_wins
    
    else:
        pass        

In [156]:
def total_and_wins_for_lane_matchup2(iterable_games_data):
    """
    Calls and aggregates results of lane_matchups() over 
    iterable collection of games data.
    Returns two counter dicts
    """
    lane_total_vs=Counter()
    lane_wins_vs=Counter()

    for game in iterable_games_data:
        try:
            lane_matchup, lane_matchup_wins = lane_matchups2(game)
            lane_total_vs+=lane_matchup
            lane_wins_vs+=lane_matchup_wins
        except:
            pass
        
    return lane_total_vs,lane_wins_vs

In [186]:

total, wins = total_and_wins_for_lane_matchup2(coll.find().limit(200))

In [ ]:
for x in wins.keys():
    if wins[x]<14:
        del wins[x]

In [184]:
lane_prob={}
for x in wins.keys():
    lane_prob[x]=wins[x]/float(total[x])

In [182]:
sorted(wins.values(), reverse=True)

[33,
 30,
 27,
 26,
 24,
 22,
 21,
 20,
 20,
 20,
 20,
 20,
 19,
 19,
 19,
 19,
 18,
 18,
 18,
 18,
 18,
 18,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 14,
 14,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]

In [185]:
sorted(lane_prob.values(),reverse=True)

[0.7727272727272727,
 0.7272727272727273,
 0.7272727272727273,
 0.7,
 0.6956521739130435,
 0.6818181818181818,
 0.673469387755102,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.6551724137931034,
 0.6538461538461539,
 0.6521739130434783,
 0.64,
 0.64,
 0.6363636363636364,
 0.6296296296296297,
 0.627906976744186,
 0.625,
 0.6206896551724138,
 0.6086956521739131,
 0.6,
 0.6,
 0.59375,
 0.5925925925925926,
 0.5833333333333334,
 0.5769230769230769,
 0.5769230769230769,
 0.5714285714285714,
 0.5652173913043478,
 0.5588235294117647,
 0.5555555555555556,
 0.5555555555555556,
 0.5555555555555556,
 0.5555555555555556,
 0.5555555555555556,
 0.5526315789473685,
 0.5517241379310345,
 0.53125,
 0.53125,
 0.5238095238095238,
 0.5142857142857142,
 0.5135135135135135,
 0.5,
 0.5,
 0.4864864864864865,
 0.4857142857142857,
 0.47619047619047616,
 0.46875,
 0.46875,
 0.4473684210526316,
 0.4444444444444444,
 0.4444444444444444,
 0.4411764705882353,
 0.43478260869565

In [128]:
for x in coll.find().limit(20):
    print check_for_normal_lanes(x)

True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
